##Import Libraries

In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

## Launch Spark Context

In [2]:
import findspark
findspark.init()
print findspark.find()
import pyspark
conf = (pyspark.SparkConf()
    .setMaster('local')
    .setAppName('pyspark')
    .set("spark.executor.memory", "4g"))
sc = pyspark.SparkContext(conf=conf)

/usr/local/opt/apache-spark/libexec


## Check child execution processes

In [3]:
import sys
rdd = sc.parallelize(xrange(10),10)
rdd.map(lambda x: sys.version).collect()

['2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x8

###SQL context for data frames

In [4]:
from pyspark.sql import SQLContext, Row
from pyspark.sql.types import *
sqlsc=SQLContext(sc)

In [5]:
#'''
# METHOD 1: convert csv file to pandas dataframe then read ino spark
colnames=['bounding_box','city','country','text','tid','timestamp','uid']
pandas_df = pd.read_csv('./tweets.csv', delimiter=',', skiprows=0, names=colnames, usecols=[1,2,3,4,5,6,7])
pandas_df.head(5)
alltweets_df = sqlsc.createDataFrame(pandas_df)
alltweets_df.printSchema()
alltweets_df.show(5)
#ustweets_df = alltweets_df[alltweets_df['country'] == "United States"]
ustweets_df = alltweets_df.filter(alltweets_df['country'] == "United States")
print "Count of All tweets =", alltweets_df.count()
print "Count of US tweets:", ustweets_df.count()
ustweets_df.show(5)
pandas_df.country.head()
#'''

root
 |-- bounding_box: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- text: string (nullable = true)
 |-- tid: string (nullable = true)
 |-- timestamp: double (nullable = true)
 |-- uid: double (nullable = true)

+--------------------+----------------+-------------+--------------------+------------------+--------------------+----------------+
|        bounding_box|            city|      country|                text|               tid|           timestamp|             uid|
+--------------------+----------------+-------------+--------------------+------------------+--------------------+----------------+
|{u'type': u'Polyg...|Huntington Beach|United States|             In need|       of back rub|6.693751389036666...|1.44842644416E12|
|{u'type': u'Polyg...|    Wheelersburg|United States|And I'm always ti...|669375138958307328|   1.448426444173E12|   3.165522051E9|
|{u'type': u'Polyg...|        Cascavel|       Brasil|    @DudaDalmas

0    United States
1    United States
2           Brasil
3    United States
4           México
Name: country, dtype: object

In [6]:
# METHOD 2: read csv directly to spark RDD then creat spark dataframe from it
alltweets_rdd = sc.textFile('./tweets.csv')
alltweets_rdd = alltweets_rdd.map(lambda line: line.split(","))
print "Total number of tweets: ", alltweets_rdd.count()
alltweets_rdd.take(2)

Total number of tweets:  37519


[[u'0',
  u'"{u\'type\': u\'Polygon\'',
  u" u'coordinates': [[[-118.082615",
  u' 33.628991]',
  u' [-118.082615',
  u' 33.756093]',
  u' [-117.91485',
  u' 33.756093]',
  u' [-117.91485',
  u' 33.628991]]]}"',
  u'Huntington Beach',
  u'United States',
  u'In need',
  u' of back rub',
  u'669375138903666692',
  u'1448426444160',
  u'1611125173'],
 [u'1',
  u'"{u\'type\': u\'Polygon\'',
  u" u'coordinates': [[[-82.87387",
  u' 38.688052]',
  u' [-82.87387',
  u' 38.76256]',
  u' [-82.811927',
  u' 38.76256]',
  u' [-82.811927',
  u' 38.688052]]]}"',
  u'Wheelersburg',
  u'United States',
  u"And I'm always tired but never of you",
  u'669375138958307328',
  u'1448426444173',
  u'3165522051']]

In [7]:
alltweets_rdd = sc.textFile('./tweets.csv')
alltweets_rdd = alltweets_rdd.map(lambda line: line.split(","))
header = alltweets_rdd.first()
#alltweets_rdd = alltweets_rdd.filter(lambda line:line != header)
#alltweets_df = alltweets_rdd.map(lambda line: Row(bounding_box=line[1:10], city=line[10], country=line[11], 
#                              text=line[12:-3], tid=line[-3], timestamp=line[-2], uid=line[-1])).toDF()
alltweets_df0 = alltweets_rdd.map(lambda line: Row(bounding_box=line[1:10], city=line[10], country=line[11], 
                              text=line[12:-3], tid=line[-3], timestamp=line[-2], uid=line[-1]))
alltweets_df = sqlsc.createDataFrame(alltweets_df0)
#alltweets_df.registerTempTable("alltweets_df")
alltweets_df.printSchema()

print type(alltweets_rdd)
print alltweets_rdd.count()
print type(alltweets_df)

root
 |-- bounding_box: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- text: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- tid: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- uid: string (nullable = true)

<class 'pyspark.rdd.PipelinedRDD'>
37519
<class 'pyspark.sql.dataframe.DataFrame'>


In [8]:
# RDD actions like count() or take() do not seem to work 
# for the alltweets_df object. Is it because dataset is too big?
# Not sure why the following return an IndexError: "list index out of range"  
#alltweets_df.count()
#alltweets_df.take(2)
#alltweets_df.show(5)

In [9]:
#alltweets_df.head()
alltweets_df.first()

Row(bounding_box=[u'"{u\'type\': u\'Polygon\'', u" u'coordinates': [[[-118.082615", u' 33.628991]', u' [-118.082615', u' 33.756093]', u' [-117.91485', u' 33.756093]', u' [-117.91485', u' 33.628991]]]}"'], city=u'Huntington Beach', country=u'United States', text=[u'In need', u' of back rub'], tid=u'669375138903666692', timestamp=u'1448426444160', uid=u'1611125173')

In [10]:
alltweets_df.select("text").first()

Row(text=[u'In need', u' of back rub'])